### Score a MotorProgram

In [ ]:
import importlib

import numpy as np

import torch
from torch.autograd import Variable

import pyro
import pyro.distributions as dist

import pandas as pd

import matlab.engine

In [ ]:
# My classes
import MotorProgram as mpr
import Stroke as stk
import CPD as cpd
import Library as lbr
import ScoreMP as smp
import MyUtils as myu

In [ ]:
# Reload my class modules quickly (avoids restarting kernel)
importlib.reload(mpr);
importlib.reload(stk);
importlib.reload(cpd);
importlib.reload(lbr);
importlib.reload(smp);
importlib.reload(myu);

In [ ]:
# Connect to existing matlab session (requires running 'matlab.engine.shareEngine' in matlab)
eng = matlab.engine.connect_matlab()  

# Or, start new matlab session
#eng = matlab.engine.start_matlab()

In [ ]:
# Return motorprogram from bottom-up process in matlab engine
mp_m = eng.parse2python()

# Load distr/param library from matlab engine
lib_m = eng.loadlib2py()


In [ ]:
# Copy matlab data into MotorProgram and Library objects

# initial with mp.ns: # of strokes
mp = mpr.MotorProgram(int(mp_m['ns'])) 

# mp.S: array of Stroke objects
for i in range(0,mp.ns):
    mp.S.append(stk.Stroke())
    mp.S[i].R = myu.mat2tensor(mp_m['S'+str(i+1)+'_R'])
    mp.S[i].ids = myu.mat2tensor(mp_m['S'+str(i+1)+'_ids'])
    mp.S[i].invscales_type = myu.mat2tensor(mp_m['S'+str(i+1)+'_invscales_type'])
    mp.S[i].shapes_type = myu.mat2tensor(mp_m['S1_shapes_type'])
    mp.S[i].nsub = mp_m['S'+str(i+1)+'_nsub']
    mp.S[i].motor = myu.mat2tensor(mp_m['S'+str(i+1)+'_motor'])
    mp.S[i].motor_spline = myu.mat2tensor(mp_m['S'+str(i+1)+'_motor_spline'])
    
# lib.pkappa: probability of each number of strokes (starts at 1)
lib = lbr.Library()
lib.pkappa = myu.mat2tensor(lib_m['pkappa'])
lib.pmat_nsub = myu.mat2tensor(lib_m['pmatnsub'])
lib.N = lib_m['N']

In [ ]:
# Score a MotorProgram
score = smp.scoreMP(mp, lib)

print(score)